# Delta Lake - Cidades Brasileiras

Este notebook demonstra a criação, leitura e manipulação de uma tabela Delta usando o dataset `cidades_brasileiras.csv`.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

from delta import *

import logging

logging.getLogger("py4j").setLevel(logging.DEBUG)

In [2]:
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

25/04/22 18:46:00 WARN Utils: Your hostname, edsatc resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/04/22 18:46:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ed/.cache/pypoetry/virtualenvs/eng-dados-spark-EC3tKTXJ-py3.11/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ed/.ivy2/cache
The jars for the packages stored in: /home/ed/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fa306b15-e51c-4707-9058-01d49e233a58;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 194ms :: artifacts dl 11ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0

In [3]:
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("cidade", StringType(), True),
    StructField("estado", StringType(), True),
    StructField("sigla", StringType(), True),
    StructField("ibge", IntegerType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True)
])

NameError: name 'DoubleType' is not defined

In [ ]:
df = spark.read.csv("../data/cidades_brasileiras.csv", header=True, schema=schema)
df.show(5)

In [ ]:
df.write.format("delta").mode("overwrite").save("../output/delta/cidades")
spark.sql("DROP TABLE IF EXISTS cidades_delta")
spark.sql("CREATE TABLE cidades_delta USING DELTA LOCATION '../output/delta/cidades'")

In [ ]:
from pyspark.sql import Row

nova_cidade = [Row(id=9999, cidade="Nova Esperança", estado="Paraná", sigla="PR", ibge=9999999, latitude=-23.5, longitude=-51.5)]
nova_df = spark.createDataFrame(nova_cidade)
nova_df.write.format("delta").mode("append").save("../output/delta/cidades")

In [ ]:
spark.sql("UPDATE cidades_delta SET cidade = 'Cidade Atualizada' WHERE id = 9999")

In [ ]:
spark.sql("DELETE FROM cidades_delta WHERE id = 9999")

In [ ]:
df_final = spark.read.format("delta").load("../output/delta/cidades")
df_final.show(5)